In [1]:
from data import get_loader, test_dataset
from options import opt
import torch.backends.cudnn as cudnn
import logging
from tensorboardX import SummaryWriter
from utils import clip_gradient, adjust_lr
from models.BBSNet_model import BBSNetChannelSpatialAttention, BBSNetTransformerAttention
from torchvision.utils import make_grid
from datetime import datetime
import numpy as np
import os
import torch
import torch.nn.functional as F
import sys
sys.path.append('./models')

from BBSNet_train import *
from BBSNet_test import *
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

# Train

In [2]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0
load data...
Start train...
x_rgb: torch.Size([1, 64, 88, 88])
r: torch.Size([1, 64, 88, 88])
r_cross: torch.Size([1, 64, 88, 88])
cat: torch.Size([1, 128, 88, 88])
g_r: torch.Size([1, 64, 88, 88])
x_rgb: torch.Size([1, 256, 88, 88])
r: torch.Size([1, 256, 88, 88])
r_cross: torch.Size([1, 256, 88, 88])
cat: torch.Size([1, 512, 88, 88])
g_r: torch.Size([1, 256, 88, 88])
x_rgb: torch.Size([1, 512, 44, 44])
r: torch.Size([1, 512, 44, 44])
r_cross: torch.Size([1, 512, 44, 44])
cat: torch.Size([1, 1024, 44, 44])
g_r: torch.Size([1, 512, 44, 44])
x_rgb: torch.Size([1, 1024, 22, 22])
r: torch.Size([1, 1024, 22, 22])
r_cross: torch.Size([1, 1024, 22, 22])
cat: torch.Size([1, 2048, 22, 22])
g_r: torch.Size([1, 1024, 22, 22])
x_rgb: torch.Size([1, 2048, 11, 11])
r: torch.Size([1, 2048, 11, 11])
r_cross: torch.Size([1, 2048, 11, 11])
cat: torch.Size([1, 4096, 11, 11])
g_r: torch.Size([1, 2048, 11, 11])
Keyboard Interrupt: save model and exit.
save checkpoints successfully!


KeyboardInterrupt: 

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [ ]:
# test(method='BBSNetChannelSpatialAttention')

In [ ]:
# evaluate()

In [ ]:
draw_curve()